In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Attack_Prediction.csv')

In [2]:
columns=df.columns

for column in columns:
    if df[column].dtype=="int32":
        df[column]=df[column].astype("int16")
    elif df[column].dtype=="float64":
        df[column]=df[column].astype("float16")
    elif df[column].dtype=="object":
        df[column]=df[column].astype("category")

In [3]:
df['Sex']= df['Sex'].map({'Female': 0, 'Male': 1})
df['Sex']= pd.to_numeric(df['Sex'])

df['Diet']= df['Diet'].map({'Healthy': 0, 'Average': 1, 'Unhealthy':2})
df['Diet']= pd.to_numeric(df['Diet'])

df[['HBP', 'LBP']]= df['Blood Pressure'].str.split('/', expand= True)
df['HBP']= pd.to_numeric(df['HBP'])
df['LBP']= pd.to_numeric(df['LBP'])

df['Diabetes'] = df['Diabetes'].map({0: 1, 1: 0})

df['Exercise Hours Per Week']= round(df['Exercise Hours Per Week'], 0)

df['Sedentary Hours Per Day']= round(df['Sedentary Hours Per Day'], 0)

df['Income']= round(df['Income'], 0)

df['BMI']= round(df['BMI'], 0)

df = df.drop(columns=['Patient ID', 'Blood Pressure', 'Country', 'Continent', 'Hemisphere'])

In [4]:
X= df.drop(['Heart Attack Risk'], axis= 1)
y= df['Heart Attack Risk']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

from sklearn.preprocessing import RobustScaler
scaler= RobustScaler()
scaler.fit(X_train)

X_train= scaler.transform(X_train)
X_test= scaler.transform(X_test)

In [6]:
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans

sm= ClusterCentroids(random_state= 15, estimator= KMeans(n_init= 10))
tl= SMOTE(random_state= 15)

X_sm, y_sm= sm.fit_resample(X_train, y_train)
X_sm_tl, y_sm_tl= tl.fit_resample(X_sm, y_sm)

In [7]:
from sklearn.linear_model import Lasso

lasso= Lasso(alpha= 0.01)
lasso.fit(X_sm_tl, y_sm_tl)

Lasso(alpha=0.01)

In [8]:
selected_features= np.where(lasso.coef_ != 0)[0]
X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

print(f"Number of features selected: {len(selected_features)}")

Number of features selected: 2


In [9]:
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred= model.predict(X_test)
    y_pred_prob= model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None

    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred, zero_division=0))
    print('Precision:', precision_score(y_test, y_pred, zero_division=0))
    print('Accuracy:', accuracy_score(y_test, y_pred))
    if y_pred_prob is not None:
        print('AUC:', roc_auc_score(y_test, y_pred_prob))
    print('-' * 50)

--- LogisticRegression ---

In [10]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression(random_state= 15)

train_and_evaluate_model(lr, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[650 475]
 [338 290]]
              precision    recall  f1-score   support

           0       0.66      0.58      0.62      1125
           1       0.38      0.46      0.42       628

    accuracy                           0.54      1753
   macro avg       0.52      0.52      0.52      1753
weighted avg       0.56      0.54      0.54      1753

Precision: 0.3790849673202614
Accuracy: 0.5362236166571591
AUC: 0.5153949044585987
--------------------------------------------------


--- DecisionTreeClassifier ---

In [11]:
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier(random_state=15)

train_and_evaluate_model(dt, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[ 121 1004]
 [  64  564]]
              precision    recall  f1-score   support

           0       0.65      0.11      0.18      1125
           1       0.36      0.90      0.51       628

    accuracy                           0.39      1753
   macro avg       0.51      0.50      0.35      1753
weighted avg       0.55      0.39      0.30      1753

Precision: 0.3596938775510204
Accuracy: 0.3907586993725043
AUC: 0.5153949044585987
--------------------------------------------------


--- Tuned - DecisionTreeClassifier ---

In [12]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt= GridSearchCV(estimator= dt, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_dt.best_params_)
print("Best Precision Score:", gs_dt.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': 20, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best Precision Score: 0.5997437966358685


In [13]:
tuned_dt= gs_dt.best_estimator_
train_and_evaluate_model(tuned_dt, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[ 121 1004]
 [  64  564]]
              precision    recall  f1-score   support

           0       0.65      0.11      0.18      1125
           1       0.36      0.90      0.51       628

    accuracy                           0.39      1753
   macro avg       0.51      0.50      0.35      1753
weighted avg       0.55      0.39      0.30      1753

Precision: 0.3596938775510204
Accuracy: 0.3907586993725043
AUC: 0.5153949044585987
--------------------------------------------------


--- KNeighborsClassifier ---

In [14]:
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier()
train_and_evaluate_model(knn, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[1125    0]
 [ 628    0]]
              precision    recall  f1-score   support

           0       0.64      1.00      0.78      1125
           1       0.00      0.00      0.00       628

    accuracy                           0.64      1753
   macro avg       0.32      0.50      0.39      1753
weighted avg       0.41      0.64      0.50      1753

Precision: 0.0
Accuracy: 0.6417569880205363
AUC: 0.5
--------------------------------------------------


--- Tuned - KNeighborsClassifier ---

In [15]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn= GridSearchCV(estimator= knn, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn.fit(X_sm_tl, y_sm_tl)
print("Best Parameters:", gs_knn.best_params_)
print("Best Precision Score:", gs_knn.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 38, 'weights': 'uniform'}
Best Precision Score: 0.531881225382125


In [16]:
tuned_knn= gs_knn.best_estimator_
train_and_evaluate_model(tuned_knn, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[1125    0]
 [ 628    0]]
              precision    recall  f1-score   support

           0       0.64      1.00      0.78      1125
           1       0.00      0.00      0.00       628

    accuracy                           0.64      1753
   macro avg       0.32      0.50      0.39      1753
weighted avg       0.41      0.64      0.50      1753

Precision: 0.0
Accuracy: 0.6417569880205363
AUC: 0.5
--------------------------------------------------


--- GaussianNB ---

In [17]:
from sklearn.naive_bayes import GaussianNB
nb= GaussianNB()
train_and_evaluate_model(nb, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[650 475]
 [338 290]]
              precision    recall  f1-score   support

           0       0.66      0.58      0.62      1125
           1       0.38      0.46      0.42       628

    accuracy                           0.54      1753
   macro avg       0.52      0.52      0.52      1753
weighted avg       0.56      0.54      0.54      1753

Precision: 0.3790849673202614
Accuracy: 0.5362236166571591
AUC: 0.5153949044585987
--------------------------------------------------


--- SVM ---

In [18]:
from sklearn.svm import SVC
svc= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
train_and_evaluate_model(svc, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[ 121 1004]
 [  64  564]]
              precision    recall  f1-score   support

           0       0.65      0.11      0.18      1125
           1       0.36      0.90      0.51       628

    accuracy                           0.39      1753
   macro avg       0.51      0.50      0.35      1753
weighted avg       0.55      0.39      0.30      1753

Precision: 0.3596938775510204
Accuracy: 0.3907586993725043
AUC: 0.5231726822363766
--------------------------------------------------


--- Random Forest ---

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(random_state=15)
train_and_evaluate_model(rf, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[ 121 1004]
 [  64  564]]
              precision    recall  f1-score   support

           0       0.65      0.11      0.18      1125
           1       0.36      0.90      0.51       628

    accuracy                           0.39      1753
   macro avg       0.51      0.50      0.35      1753
weighted avg       0.55      0.39      0.30      1753

Precision: 0.3596938775510204
Accuracy: 0.3907586993725043
AUC: 0.5153949044585987
--------------------------------------------------


In [20]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf= GridSearchCV(estimator= rf, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf.fit(X_sm_tl, y_sm_tl)
print("Best Parameters:", gs_tuned_rf.best_params_)
print("Best Precision Score:", gs_tuned_rf.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 300}
Best Precision Score: 0.6477451699227709


In [21]:
tuned_rf= gs_tuned_rf.best_estimator_
train_and_evaluate_model(tuned_rf, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[ 121 1004]
 [  64  564]]
              precision    recall  f1-score   support

           0       0.65      0.11      0.18      1125
           1       0.36      0.90      0.51       628

    accuracy                           0.39      1753
   macro avg       0.51      0.50      0.35      1753
weighted avg       0.55      0.39      0.30      1753

Precision: 0.3596938775510204
Accuracy: 0.3907586993725043
AUC: 0.5153949044585987
--------------------------------------------------


--- AdaBoost ---

In [22]:
from sklearn.ensemble import AdaBoostClassifier
ada= AdaBoostClassifier(random_state=15)
train_and_evaluate_model(ada, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[350 775]
 [193 435]]
              precision    recall  f1-score   support

           0       0.64      0.31      0.42      1125
           1       0.36      0.69      0.47       628

    accuracy                           0.45      1753
   macro avg       0.50      0.50      0.45      1753
weighted avg       0.54      0.45      0.44      1753

Precision: 0.359504132231405
Accuracy: 0.4478037649743297
AUC: 0.5153949044585987
--------------------------------------------------


In [23]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada= GridSearchCV(estimator= ada, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada.fit(X_sm_tl, y_sm_tl)
print("Best Parameters:", gs_ada.best_params_)
print("Best Precision Score:", gs_ada.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.6490728181226774


In [24]:
tuned_ada= gs_ada.best_estimator_
train_and_evaluate_model(tuned_ada, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[350 775]
 [193 435]]
              precision    recall  f1-score   support

           0       0.64      0.31      0.42      1125
           1       0.36      0.69      0.47       628

    accuracy                           0.45      1753
   macro avg       0.50      0.50      0.45      1753
weighted avg       0.54      0.45      0.44      1753

Precision: 0.359504132231405
Accuracy: 0.4478037649743297
AUC: 0.5153949044585987
--------------------------------------------------


--- GradientBoosting ---

In [25]:
from sklearn.ensemble import GradientBoostingClassifier
grb= GradientBoostingClassifier(random_state=15)
train_and_evaluate_model(grb, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[ 121 1004]
 [  64  564]]
              precision    recall  f1-score   support

           0       0.65      0.11      0.18      1125
           1       0.36      0.90      0.51       628

    accuracy                           0.39      1753
   macro avg       0.51      0.50      0.35      1753
weighted avg       0.55      0.39      0.30      1753

Precision: 0.3596938775510204
Accuracy: 0.3907586993725043
AUC: 0.5153949044585987
--------------------------------------------------


In [26]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb= GridSearchCV(estimator= grb, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_grb.best_params_)
print("Best Precision Score:", gs_grb.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.2}
Best Precision Score: 0.6642793434216976


In [27]:
tuned_grb= gs_grb.best_estimator_
train_and_evaluate_model(tuned_grb, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[ 121 1004]
 [  64  564]]
              precision    recall  f1-score   support

           0       0.65      0.11      0.18      1125
           1       0.36      0.90      0.51       628

    accuracy                           0.39      1753
   macro avg       0.51      0.50      0.35      1753
weighted avg       0.55      0.39      0.30      1753

Precision: 0.3596938775510204
Accuracy: 0.3907586993725043
AUC: 0.5153949044585987
--------------------------------------------------


--- XGB ---

In [28]:
from xgboost import XGBClassifier
xgb= XGBClassifier(random_state=15)
train_and_evaluate_model(xgb, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[ 121 1004]
 [  64  564]]
              precision    recall  f1-score   support

           0       0.65      0.11      0.18      1125
           1       0.36      0.90      0.51       628

    accuracy                           0.39      1753
   macro avg       0.51      0.50      0.35      1753
weighted avg       0.55      0.39      0.30      1753

Precision: 0.3596938775510204
Accuracy: 0.3907586993725043
AUC: 0.5153949044585987
--------------------------------------------------


In [29]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb= GridSearchCV(estimator= xgb, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_xgb.best_params_)
print("Best Precision Score:", gs_xgb.best_score_)

tuned_xgb= gs_xgb.best_estimator_
train_and_evaluate_model(tuned_xgb, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.671251297680209
[[ 121 1004]
 [  64  564]]
              precision    recall  f1-score   support

           0       0.65      0.11      0.18      1125
           1       0.36      0.90      0.51       628

    accuracy                           0.39      1753
   macro avg       0.51      0.50      0.35      1753
weighted avg       0.55      0.39      0.30      1753

Precision: 0.3596938775510204
Accuracy: 0.3907586993725043
AUC: 0.5153949044585987
--------------------------------------------------


--- LGBM ---

In [30]:
from lightgbm import LGBMClassifier
lgm= LGBMClassifier(random_state=15)
train_and_evaluate_model(lgm, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[LightGBM] [Info] Number of positive: 2511, number of negative: 2511
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004689 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 20
[LightGBM] [Info] Number of data points in the train set: 5022, number of used features: 2
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

In [31]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm= GridSearchCV(estimator= lgm, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_lgm.best_params_)
print("Best Precision Score:", gs_lgm.best_score_)

tuned_lgm= gs_lgm.best_estimator_
train_and_evaluate_model(tuned_lgm, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Info] Number of positive: 2511, number of negative: 2511
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001584 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2253
[LightGBM] [Info] Number of data points in the train set: 5022, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Best Parameters: {'learning_rate': 0.1, 'min_data_in_leaf': 30, 'num_leaves': 31, 'reg_alpha': 0.5}
Best Precision Score: 0.6695895316370789
[LightGBM] [Warning] min_data_in_leaf is set=30, min_

--- CatBoost ---

In [32]:
from catboost import CatBoostClassifier
cat= CatBoostClassifier(learning_rate= 0.1, depth= 6, iterations= 100, random_state= 15, verbose= 0)
train_and_evaluate_model(cat, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[ 121 1004]
 [  64  564]]
              precision    recall  f1-score   support

           0       0.65      0.11      0.18      1125
           1       0.36      0.90      0.51       628

    accuracy                           0.39      1753
   macro avg       0.51      0.50      0.35      1753
weighted avg       0.55      0.39      0.30      1753

Precision: 0.3596938775510204
Accuracy: 0.3907586993725043
AUC: 0.5153949044585987
--------------------------------------------------


In [33]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat= GridSearchCV(estimator= cat, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_cat.best_params_)
print("Best Precision Score:", gs_cat.best_score_)

tuned_cat= gs_cat.best_estimator_
train_and_evaluate_model(tuned_cat, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best Parameters: {'depth': 5, 'iterations': 100, 'learning_rate': 0.1}
Best Precision Score: 0.6641814124189656
[[ 121 1004]
 [  64  564]]
              precision    recall  f1-score   support

           0       0.65      0.11      0.18      1125
           1       0.36      0.90      0.51       628

    accuracy                           0.39      1753
   macro avg       0.51      0.50      0.35      1753
weighted avg       0.55      0.39      0.30      1753

Precision: 0.3596938775510204
Accuracy: 0.3907586993725043
AUC: 0.5153949044585987
--------------------------------------------------


In [34]:
y_pred_lr= lr.predict(X_sm_tl_test_selected)
y_pred_prob_lr= lr.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_dt= dt.predict(X_sm_tl_test_selected)
y_pred_prob_dt= dt.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_tuned_dt= tuned_dt.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_dt= tuned_dt.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_knn= knn.predict(X_sm_tl_test_selected)
y_pred_prob_knn= knn.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_tuned_knn= tuned_knn.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_knn= tuned_knn.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_nb= nb.predict(X_sm_tl_test_selected)
y_pred_prob_nb= nb.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_svc= svc.predict(X_sm_tl_test_selected)
y_pred_prob_svc= svc.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_rf= rf.predict(X_sm_tl_test_selected)
y_pred_prob_rf= rf.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_tuned_rf= tuned_rf.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_rf= tuned_rf.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_ada= ada.predict(X_sm_tl_test_selected)
y_pred_prob_ada= ada.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_tuned_ada= tuned_ada.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_ada= tuned_ada.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_grb= grb.predict(X_sm_tl_test_selected)
y_pred_prob_grb= grb.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_tuned_grb= tuned_grb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_grb= tuned_grb.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_xgb= xgb.predict(X_sm_tl_test_selected)
y_pred_prob_xgb= xgb.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_tuned_xgb= tuned_xgb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_xgb= tuned_xgb.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_lgm= lgm.predict(X_sm_tl_test_selected)
y_pred_prob_lgm= lgm.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_tuned_lgm= tuned_lgm.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_lgm= tuned_lgm.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_cat= cat.predict(X_sm_tl_test_selected)
y_pred_prob_cat= cat.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_tuned_cat= tuned_cat.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_cat= tuned_cat.predict_proba(X_sm_tl_test_selected)[:,1]

[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30


--- Visualisation ---

In [35]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat, zero_division= 0)
                  }

be_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
be_precision= be_precision.sort_values(by= 'Precision Score', ascending=False)
print(be_precision)

                                    Model  Precision Score
0          Logistic Regression Precision:         0.379085
5                   GaussianNB Precision:         0.379085
11            GradientBoosting Precision:         0.359694
17                    CatBoost Precision:         0.359694
16                  Tuned LGBM Precision:         0.359694
15                        LGBM Precision:         0.359694
14                   Tuned XGB Precision:         0.359694
13                         XGB Precision:         0.359694
12      Tuned GradientBoosting Precision:         0.359694
18              Tuned CatBoost Precision:         0.359694
1                Decision Tree Precision:         0.359694
8          Tuned Random Forest Precision:         0.359694
7                Random Forest Precision:         0.359694
6                          SVM Precision:         0.359694
2          Tuned Decision Tree Precision:         0.359694
10              Tuned AdaBoost Precision:         0.3595

In [36]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat)
                  }

be_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
be_accuracy= be_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(be_accuracy)

                                   Model  Accuracy Score
3         KNeighborsClassifier Accuracy:        0.641757
4   Tuned KNeighborsClassifier Accuracy:        0.641757
0          Logistic Regression Accuracy:        0.536224
5                   GaussianNB Accuracy:        0.536224
10              Tuned AdaBoost Accuracy:        0.447804
9                     AdaBoost Accuracy:        0.447804
6                          SVM Accuracy:        0.390759
7                Random Forest Accuracy:        0.390759
8          Tuned Random Forest Accuracy:        0.390759
1                Decision Tree Accuracy:        0.390759
2          Tuned Decision Tree Accuracy:        0.390759
11            GradientBoosting Accuracy:        0.390759
12      Tuned GradientBoosting Accuracy:        0.390759
13                         XGB Accuracy:        0.390759
14                   Tuned XGB Accuracy:        0.390759
15                        LGBM Accuracy:        0.390759
16                  Tuned LGBM 

In [37]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat)
                  }

be_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
be_auc= be_auc.sort_values(by= 'AUC Score', ascending=False)
print(be_auc)

                              Model  AUC Score
6                          SVM AUC:   0.523173
0          Logistic Regression AUC:   0.515395
10              Tuned AdaBoost AUC:   0.515395
17                    CatBoost AUC:   0.515395
16                  Tuned LGBM AUC:   0.515395
15                        LGBM AUC:   0.515395
14                   Tuned XGB AUC:   0.515395
13                         XGB AUC:   0.515395
12      Tuned GradientBoosting AUC:   0.515395
11            GradientBoosting AUC:   0.515395
9                     AdaBoost AUC:   0.515395
1                Decision Tree AUC:   0.515395
8          Tuned Random Forest AUC:   0.515395
7                Random Forest AUC:   0.515395
5                   GaussianNB AUC:   0.515395
2          Tuned Decision Tree AUC:   0.515395
18              Tuned CatBoost AUC:   0.515395
4   Tuned KNeighborsClassifier AUC:   0.500000
3         KNeighborsClassifier AUC:   0.500000
